In [14]:
import dotenv, os, requests
import pandas as pd
import numpy as np
import pandasql as ps

In [15]:
dotenv.load_dotenv('ref/.env')

True

In [16]:
league_id = os.environ.get('tds')
sports_data_key = os.environ.get('sports_data')

In [17]:
base_api_url = 'https://api.sleeper.app/v1/'
sports_data_url = 'https://api.sportsdata.io/v3/nfl/stats/json/Injuries/2024'

In [18]:
league_info = requests.get(f'{base_api_url}league/{league_id}').json()

In [19]:
league_info = {
    'league_name' : league_info['name']
    ,'start_week' : league_info['settings']['start_week']
    ,'playoff_start_week' : league_info['settings']['playoff_week_start']
    ,'roster_slots' : [pos for pos in league_info['roster_positions'] if pos != 'BN']
}

In [62]:
def get_injuries_df():
    injuries_resp = [requests.get(f'{sports_data_url}/{wk}?key={sports_data_key}').json() for wk in range(league_info['start_week'], league_info['playoff_start_week'])]
    injuries = []
    for wk in injuries_resp:
        for injury in wk:
            injuries.append({'full_name' : injury['Name'], 'week' : injury['Week'], 'team' : injury['Team']})
    return pd.DataFrame(injuries)

In [63]:
injuries_df = get_injuries_df()

In [23]:
def get_weekly_matchups_dfs():
    plyr_wkly_points_dicts = []
    wkly_matchups_dicts = []
    wkly_matchups = [requests.get(f'{base_api_url}league/{league_id}/matchups/{wk}').json() for wk in range(league_info['start_week'], league_info['playoff_start_week'])]
    week_num = 0
    for wk in wkly_matchups:
        week_num += 1
        for matchup in wk:
            wkly_matchups_dicts.append({'roster_id' : matchup['roster_id'], 'week' : week_num, 'matchup_id' : matchup['matchup_id'], 'ttl_points' : matchup['points']})
            for player in matchup['players']:
                plyr_wkly_points_dicts.append({'roster_id': matchup['roster_id'], 'player_id' : player, 'week': week_num, 'is_starter' : True if player in matchup['starters'] else False, 'points' : matchup['players_points'][player]})
    return (pd.DataFrame(wkly_matchups_dicts), pd.DataFrame(plyr_wkly_points_dicts))

In [24]:
def get_weekly_transactions_dfs():
    wkly_trnsctns = [requests.get(f'{base_api_url}league/{league_id}/transactions/{wk}').json() for wk in range(league_info['start_week'], league_info['playoff_start_week'])]
    trns_dicts = []
    plyr_trns_dicts = []
    week_num = 0
    player_trns_typs = ['add', 'drop']
    for wk in wkly_trnsctns:
        week_num += 1
        for trns in wk:
            for trns_typ in player_trns_typs:
                if trns[f'{trns_typ}s']:
                    for player in trns[f'{trns_typ}s'].keys():
                        plyr_trns_dicts.append({
                            'week' : week_num
                            ,'player_id' : player
                            ,'roster_id' : trns[f'{trns_typ}s'][player]
                            ,'add_drop' : trns_typ
                            ,'trns_id' : trns['transaction_id']
                            ,'create_tms' : trns['created']
                        })
            for roster in trns['roster_ids']:
                trns_dicts.append({
                    'roster_id' : roster
                    ,'trns_typ' : trns['type']
                    ,'trns_status' : trns['status']
                    ,'trns_id' : trns['transaction_id']
                    ,'create_tms' : trns['created']
                })
    return (pd.DataFrame(trns_dicts), pd.DataFrame(plyr_trns_dicts))

In [25]:
def get_roster_id_by_team_df():
    users_df = pd.DataFrame(requests.get(f'{base_api_url}league/{league_id}/users').json())[['display_name', 'user_id']]
    rosters_df = pd.DataFrame([{'roster_id' : roster['roster_id'], 'owner_id' : roster['owner_id'], 'record' : roster['metadata']['record']} for roster in requests.get(f'{base_api_url}league/{league_id}/rosters').json()])
    return users_df.merge(rosters_df, how='inner', left_on='user_id', right_on='owner_id')[['display_name', 'roster_id', 'record']]

In [158]:
def get_draft_df():
    draft_id = requests.get(f'{base_api_url}league/{league_id}/drafts').json()[0]['draft_id']
    draft_tm = requests.get(f'{base_api_url}draft/{draft_id}').json()['created']
    draft_resp = requests.get(f'{base_api_url}draft/{draft_id}/picks').json()
    return pd.DataFrame([{'player_id' : pick['player_id'], 'roster_id' : pick['roster_id'], 'round' : pick['round'], 'trns_typ' : 'draft', 'trns_status' : 'complete', 'week' : 0, 'add_drop' : 'add', 'create_tms' : draft_tm, 'trns_id' : f'{pick['draft_id']}{pick['roster_id']}{pick['round']}'} for pick in draft_resp])

In [163]:
draft_df = get_draft_df()

In [164]:
matchups_df, plyr_points_df = get_weekly_matchups_dfs()

In [165]:
trns_df, plyr_trns_df = get_weekly_transactions_dfs()

In [166]:
draft_df['create_tms'] = pd.to_datetime(draft_df.create_tms, unit='ms')
plyr_trns_df['create_tms'] = pd.to_datetime(plyr_trns_df.create_tms, unit='ms')
trns_df['create_tms'] = pd.to_datetime(trns_df.create_tms, unit='ms')

In [167]:
plyr_trns_df = pd.concat([plyr_trns_df, draft_df[['week', 'player_id', 'roster_id', 'add_drop', 'trns_id', 'create_tms']]])
trns_df = pd.concat([trns_df, draft_df[['roster_id', 'trns_typ', 'trns_status', 'trns_id', 'create_tms']]])

In [168]:
plyr_df = pd.read_json('ref/players.json', orient='index')[['first_name', 'last_name', 'team', 'position', 'player_id', 'years_exp', 'full_name']]

/var/folders/0b/_drclt_54kb8why6nm6znsnh0000gn/T/ipykernel_75588/2706744226.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  plyr_df = pd.read_json('ref/players.json', orient='index')[['first_name', 'last_name', 'team', 'position', 'player_id', 'years_exp', 'full_name']]
/var/folders/0b/_drclt_54kb8why6nm6znsnh0000gn/T/ipykernel_75588/2706744226.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  plyr_df = pd.read_json('ref/players.json', orient='index')[['first_name', 'last_nam

In [169]:
rosters_df = get_roster_id_by_team_df()

In [170]:
def get_rookie_starts_df(plyr_points_df, plyr_df, rosters_df):
    strted_df = plyr_points_df[plyr_points_df.is_starter == True].merge(plyr_df[plyr_df.years_exp == 0], on='player_id', how='inner').merge(rosters_df, on='roster_id', how='inner')[['player_id', 'display_name', 'week']]
    rookies_strtd_df = strted_df.groupby('display_name').player_id.nunique().to_frame().reset_index().rename({'player_id' : 'num_rookies_started'}, axis=1)
    rookie_strts_df = strted_df.groupby('display_name').player_id.count().to_frame().reset_index().rename({'player_id' : 'ttl_rookie_starts'}, axis=1)
    return rookies_strtd_df.merge(rookie_strts_df).sort_values(by=['ttl_rookie_starts', 'num_rookies_started'], ascending=False)

In [171]:
def get_win_streak(row):
    return max(map(len, row.split('L')))

def get_loss_streak(row):
    return max(map(len, row.split('W')))

In [172]:
rosters_df['win_streak'] = rosters_df.record.apply(get_win_streak)
rosters_df['loss_streak'] = rosters_df.record.apply(get_loss_streak)

In [173]:
def get_rstrd_plyr_to_fm_df(plyr_trns_df, trns_df):
    merged_trns_df = plyr_trns_df.merge(trns_df[trns_df.trns_status == 'complete'], how='inner').sort_values(['roster_id', 'player_id', 'create_tms'])
    merged_trns_df['plyr_actn_rnk'] = merged_trns_df.groupby(['player_id', 'roster_id'])['create_tms'].rank(ascending=True)
    merged_trns_df.loc[merged_trns_df.add_drop == 'add', 'merge_on_actn_rnk'] = merged_trns_df.plyr_actn_rnk + 1
    merged_trns_df.drop(['trns_id', 'create_tms', 'trns_status'], axis=1, inplace=True)
    merged_trns_df = merged_trns_df.merge(merged_trns_df, left_on=['merge_on_actn_rnk', 'player_id', 'roster_id'], right_on=['plyr_actn_rnk', 'player_id', 'roster_id'], how='left')
    merged_trns_df = merged_trns_df[merged_trns_df.add_drop_x == 'add']
    merged_trns_df = merged_trns_df.rename({
        'week_x' : 'week_fm'
        ,'week_y' : 'week_to'
        ,'trns_typ_x' : 'add_action'
        ,'trns_typ_y' : 'drop_action'
    }, axis=1).drop(['add_drop_x', 'add_drop_y', 'plyr_actn_rnk_x', 'plyr_actn_rnk_y', 'merge_on_actn_rnk_x', 'merge_on_actn_rnk_y'], axis=1)
    merged_trns_df['week_to'] = merged_trns_df.week_to.fillna(league_info['playoff_start_week']).astype('int')
    return merged_trns_df[(merged_trns_df.add_action != 'commissioner') | (merged_trns_df.week_fm != 1)][['roster_id', 'player_id', 'add_action', 'drop_action', 'week_fm', 'week_to']]

In [174]:
rstrd_plyr_to_fm_df = get_rstrd_plyr_to_fm_df(plyr_trns_df, trns_df)

In [72]:
def get_waiver_scores_df(rstrd_plyr_to_fm_df, plyr_points_df):
    return ps.sqldf("""
    SELECT
        rdf.display_name
        ,pdf.first_name
        ,pdf.last_name
        ,ppdf.player_id
        ,Sum(ppdf.points) AS ttl_points
        ,count(ppdf.week) AS wks_strtd
        ,Sum(ppdf.points) / Count(ppdf.week) AS avg_strt_pts
        FROM plyr_points_df ppdf
        INNER JOIN rstrd_plyr_to_fm_df rpdf
            ON rpdf.player_id = ppdf.player_id
            AND rpdf.roster_id = ppdf.roster_id
            AND ppdf.week >= rpdf.week_fm
            AND ppdf.week < rpdf.week_to
        INNER JOIN rosters_df rdf
            ON rdf.roster_id = rpdf.roster_id
        INNER JOIN plyr_df pdf
            ON pdf.player_id = ppdf.player_id
        WHERE 1=1
            AND ppdf.is_starter = True
            AND rpdf.drop_action IN ('waiver', 'free_agent')
        GROUP BY 1,2
    """)

In [73]:
def get_wkly_matchup_results_df(matchups_df, rosters_df):
    wk_median_df = pd.DataFrame(matchups_df.groupby(matchups_df.week)[['ttl_points']].median()).reset_index().rename({'ttl_points' : 'wk_median'}, axis=1)
    return ps.sqldf("""
    SELECT
        rdf.display_name
        ,l.week
        ,l.ttl_points AS pf
        ,r.ttl_points AS pa
        ,ABS(l.ttl_points - r.ttl_points) AS abs_pt_diff
        ,wmdf.wk_median AS league_median
        ,CASE
            WHEN l.ttl_points > r.ttl_points
                THEN 1
            ELSE 0
        END AS win
        ,CASE
            WHEN l.ttl_points < r.ttl_points
                THEN 1
            ELSE 0
        END AS loss
        ,CASE
            WHEN l.ttl_points = r.ttl_points
                THEN 1
            ELSE 0
        END AS tie
        ,CASE
            WHEN l.ttl_points > wmdf.wk_median
                THEN 1
            ELSE 0
        END AS median_win
        ,CASE
            WHEN l.ttl_points < wmdf.wk_median
                THEN 1
            ELSE 0
        END AS median_loss
        ,CASE
            WHEN l.ttl_points = wmdf.wk_median
                THEN 1
            ELSE 0
        END AS median_tie
        FROM matchups_df l
        INNER JOIN matchups_df r
            ON r.matchup_id = l.matchup_id
            AND r.roster_id <> l.roster_id
            AND r.week = l.week
        INNER JOIN rosters_df rdf
            ON rdf.roster_id = l.roster_id
        INNER JOIN wk_median_df wmdf
            ON wmdf.week = l.week
    """)

In [74]:
matchup_results_df = get_wkly_matchup_results_df(matchups_df, rosters_df)

In [75]:
srtd_wins_df = matchup_results_df[matchup_results_df.win == 1].sort_values('abs_pt_diff', ascending=False)
srtd_losses_df = matchup_results_df[matchup_results_df.loss == 1].sort_values('abs_pt_diff', ascending=False)

In [76]:
median_results_df = matchup_results_df.groupby('display_name')[['pf', 'pa', 'win', 'loss', 'tie', 'median_win', 'median_loss', 'median_tie']].sum().reset_index()

In [77]:
median_results_df['ttl_wins'] = median_results_df.win + median_results_df.median_win
median_results_df['ttl_losses'] = median_results_df.loss + median_results_df.median_loss
median_results_df['ttl_ties'] = median_results_df.tie + median_results_df.median_tie
median_results_df['ttl_win_perc'] = (median_results_df.ttl_wins + (median_results_df.ttl_ties / 2)) / (median_results_df.ttl_wins + median_results_df.ttl_losses + median_results_df.ttl_ties)

In [78]:
median_results_df.sort_values('ttl_win_perc', ascending=False).reset_index(drop=True)

,display_name,pf,pa,win,loss,tie,median_win,median_loss,median_tie,ttl_wins,ttl_losses,ttl_ties,ttl_win_perc
0,hatersandlosers,2017.32,1612.14,11,3,0,11,3,0,22,6,0,0.785714
1,aspear3,2024.04,1765.26,10,4,0,11,3,0,21,7,0,0.750000
2,kerrveball,1982.44,1593.68,10,4,0,9,5,0,19,9,0,0.678571
3,MadelynShaw,1726.32,1380.82,11,3,0,7,7,0,18,10,0,0.642857
4,rmasons,1883.40,1679.90,8,6,0,10,4,0,18,10,0,0.642857
5,UB400,1769.20,1887.32,6,8,0,9,5,0,15,13,0,0.535714
6,blspain003,1654.56,1728.58,8,6,0,6,8,0,14,14,0,0.500000
7,IWantSomeButts,1653.74,1807.60,5,9,0,7,7,0,12,16,0,0.428571
8,catrussell,1569.76,1829.54,5,9,0,6,8,0,11,17,0,0.392857
9,GriffDaGreat,1587.60,1672.88,4,10,0,5,9,0,9,19,0,0.321429


In [79]:
matchup_results_df[matchup_results_df.loss == 1].sort_values('pf', ascending=False)

,display_name,week,pf,pa,abs_pt_diff,league_median,win,loss,tie,median_win,median_loss,median_tie
60,rmasons,6,156.36,165.40,9.04,118.29,0,1,0,1,0,0
132,rmasons,12,142.52,153.06,10.54,119.26,0,1,0,1,0,0
165,UB400,14,135.08,147.88,12.80,130.52,0,1,0,1,0,0
51,IWantSomeButts,5,131.76,153.86,22.10,131.49,0,1,0,1,0,0
105,UB400,9,131.50,143.74,12.24,124.13,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
118,ItIsATravisty,10,75.18,117.52,42.34,116.85,0,1,0,0,1,0
44,RyJohn,4,75.04,138.60,63.56,125.45,0,1,0,0,1,0
106,ItIsATravisty,9,71.54,142.54,71.00,124.13,0,1,0,0,1,0
111,IWantSomeButts,10,69.00,152.20,83.20,116.85,0,1,0,0,1,0


In [87]:
win_avg_points_df = matchup_results_df[matchup_results_df.win == 1].groupby('display_name')[['abs_pt_diff']].mean().reset_index().sort_values(by='abs_pt_diff', ascending=True)

In [88]:
win_avg_points_df

,display_name,abs_pt_diff
4,RyJohn,7.400000
7,blspain003,20.577500
8,catrussell,20.832000
2,ItIsATravisty,22.675000
0,GriffDaGreat,32.450000
11,rmasons,37.452500
3,MadelynShaw,38.581818
6,aspear3,44.802000
10,kerrveball,44.890000
5,UB400,45.493333


In [89]:
loss_avg_points_df = matchup_results_df[matchup_results_df.loss  == 1].groupby('display_name')[['abs_pt_diff']].mean().reset_index().sort_values(by='abs_pt_diff', ascending=False)

In [90]:
loss_avg_points_df

,display_name,abs_pt_diff
9,hatersandlosers,50.540000
5,UB400,48.885000
6,aspear3,47.310000
2,ItIsATravisty,47.114000
4,RyJohn,45.418333
1,IWantSomeButts,43.700000
8,catrussell,40.437778
7,blspain003,39.773333
3,MadelynShaw,26.300000
0,GriffDaGreat,21.508000


In [186]:
rosterd_injuries = ps.sqldf("""
SELECT
    rdf.display_name
    ,pdf.player_id
    ,idf.week
    ,idf.team
    FROM injuries_df idf
    INNER JOIN plyr_df pdf
        ON pdf.full_name = idf.full_name
        AND pdf.team = idf.team
    INNER JOIN rstrd_plyr_to_fm_df rptfdf
        ON rptfdf.player_id = pdf.player_id
        AND idf.week BETWEEN rptfdf.week_fm AND rptfdf.week_to
    INNER JOIN rosters_df rdf
        ON rdf.roster_id = rptfdf.roster_id
""").drop_duplicates()

In [188]:
rosterd_injuries.groupby(['display_name'])['player_id'].nunique().reset_index().sort_values(by='player_id', ascending=False)

,display_name,player_id
5,UB400,17
11,rmasons,12
0,GriffDaGreat,9
1,IWantSomeButts,9
2,ItIsATravisty,8
3,MadelynShaw,8
4,RyJohn,7
8,catrussell,7
9,hatersandlosers,7
7,blspain003,6
